In [ ]:
import numpy as np
import scripts
import geopandas
import matplotlib.pyplot as plt
import stoclust as sc
import matplotlib.lines as ln
from tqdm import tqdm
import os

import plotly.graph_objects as go
import scipy.linalg as la
import scipy.special as sp

# Overview

In this notebook, we simply run the scripts which generate the technical coefficients $C_{ri,sj}$ for the null and empirical models. We then compute the Leontief inverse $L_{ri,sj} = \left(\mathbf{I}-\mathbf{C}\right)^{-1}_{ri,sj}$ using further scripts.

# Initialization

We generate the block matrices for the technical coefficients of the null and empirical models. These will be saved to files and must be subsequently loaded.

In [ ]:
scripts.null_blocks()
scripts.industry_blocks()

In [ ]:
gtap = scripts.GTAP()
num_reg = gtap.regions.items.size
num_commod = gtap.commodities.items.size
num_fac = gtap.factors.items.size

Use `null` as the model name for the null model. Use `industry` as the model name for the GTAP 8 data.

In [ ]:
model = 'null'
names = sc.Group(['D','M','F','C'])

T = {
    names[i]+names[j]: np.load('scripts/data/computed/blocks/'+model+'/'+names[i]+names[j]+'_T.npy')
    for i in range(len(names))
    for j in range(len(names))
}

Next, we run the Leontief inverses for the null model data and the empirical data. Since there is only one empirical outcome and many generated null model samples, this will take longer for the null model.

In [ ]:
def get_leontief(blocks,exo='C',output=False):
    for k in range(4):
        blocks[k][names.ind[exo]] = np.zeros(blocks[k][names.ind[exo]].shape)
    L = scripts.divide_and_conquer(blocks,output=output)
    return L

In [ ]:
if model == 'null':
    Ls = [[[],[],[],[]],
          [[],[],[],[]],
          [[],[],[],[]],
          [[],[],[],[]]]
    for k in tqdm(range(T['DD'].shape[0])):
        blocks = [[T[names[i]+names[j]][k] for j in range(len(names))] for i in range(len(names))]
        L = get_leontief(blocks)
        for i in range(len(names)):
            for j in range(len(names)):
                Ls[i][j].append(L[i][j])
    L = [
        [np.array(Ls[i][j]) for j in range(len(names))] for i in range(len(names))
    ]
else:
    blocks = [[T[names[i]+names[j]] for j in range(len(names))] for i in range(len(names))]
    L = get_leontief(blocks,output=True)
    
if not os.path.isdir(os.path.abspath('scripts/data/computed/leontief')):
    os.mkdir(os.path.abspath('scripts/data/computed/leontief'))
if not os.path.isdir(os.path.abspath('scripts/data/computed/leontief/'+model)):
    os.mkdir(os.path.abspath('scripts/data/computed/leontief/'+model))

for i in range(4):
    for j in range(4):
        np.save('scripts/data/computed/leontief/'+model+'/'+names[i]+names[j]+'_L.npy',L[i][j])